In [1]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
test_csv = pd.read_csv("test.csv")
train_csv = pd.read_csv("train.csv")

In [3]:
nump_test = test_csv.values
nump_train = train_csv.values

In [4]:
test_torch = torch.from_numpy(nump_test)
train_torch = torch.from_numpy(nump_train)

In [5]:
testX = test_torch[:, :-1]
testY = test_torch[:, -1]
testY = testY.reshape(-1,1)

In [6]:
trainX = train_torch[:, :-1]
trainY = train_torch[:, -1]
trainY = trainY.reshape(-1,1)

In [7]:
testX = testX.to(torch.float64)
testY = testY.to(torch.float64)
trainX = trainX.to(torch.float64)
trainY = trainY.to(torch.float64)

In [8]:
#Q1: about how many bytes does trainX consume?
dtype = trainX.dtype
print("dtype is:", dtype) # float64 is 64 bits = 8 bytes
shape = trainX.shape
print("shape is:", shape)
consumption_trainX = 1044 * 10 * 8
consumption_trainX

dtype is: torch.float64
shape is: torch.Size([1044, 10])


83520

In [9]:
#Q2: what is the biggest difference we would have any one cell if we used float16 instead of float64?
trainx_64 = trainX
trainx_16 = trainX.to(torch.float16)
new_train = torch.sub(trainx_64, trainx_16)
max_train = new_train.max()
answer_q2 = max_train.item()
answer_q2

0.0

In [10]:
#Q3: is a CUDA GPU available on your VM?
#help from: https://saturncloud.io/blog/how-to-check-whether-your-code-is-running-on-the-gpu-or-cpu/#:~:text=If%20you%27re%20using%20Python,is%20available%20and%20False%20otherwise
answer_q3 = False
if torch.cuda.is_available():
    answer_q3 = True
else:
    answer_q3 = False
answer_q3

False

In [11]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [12]:
#Q4: what is the predicted number of deaths for the first census tract?
result = testX[0] * coef.view(-1)
predicted_deaths = result.sum()
predicted_deaths = predicted_deaths.item()
predicted_deaths

9.844

In [13]:
#Q5: what is the average number of predicted deaths, over the whole testX dataset?
deaths = torch.matmul(testX, coef)    
avg_deaths = torch.mean(deaths).item()
avg_deaths

12.073632183908048

In [14]:
#Q6: first, what is y when x is a tensor containing 0.0?

def f(x):
    
    return (x**2) - (8*x) + 19
    
x = torch.tensor(0.0)
y = f(x)
float(y)

19.0

In [15]:
#Q7: what x value minimizes y?

x = torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x], lr=0.2)

for epoch in range(100):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad() 
x.item()

3.999999761581421

In [16]:
#Q8: what is the MSE (mean-square error) when we make predictions using this vector of zero coefficients?
#help from: chat6 pdf
num_samples, num_features = trainX.shape

coef_q8 = torch.zeros((num_features, 1), dtype=torch.float64)

predictions_q8 = trainX @ coef_q8

mse_q8 = torch.mean((predictions_q8 - trainY.view(-1, 1))**2).item()
mse_q8

197.8007662835249

In [17]:
#Q9: what is the MSE over the training data, using the coefficients resulting from the above training?
#help from: chat7 pdf
torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)
loss_fn = torch.nn.MSELoss()
coef_q9 = torch.zeros((trainX.shape[1], 1), dtype=torch.float64, requires_grad=True)


optimizer_q9 = torch.optim.SGD([coef_q9], lr=0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        predictions_q9 = batchX @ coef_q9
        loss_q9 = loss_fn(predictions_q9, batchY.view(-1, 1))
        
        optimizer_q9.zero_grad()
        loss_q9.backward()
        optimizer_q9.step()

with torch.no_grad():
    predictions_q9 = trainX @ coef_q9
    mse_q9 = loss_fn(predictions_q9, trainY.view(-1, 1)).item()

mse_q9

26.8113940147193

In [18]:
#Q10: what is the MSE over the test data?
#help from: chat7 pdf
torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)
loss_fn_q10 = torch.nn.MSELoss()
coef_q10 = torch.zeros((testX.shape[1], 1), dtype=torch.float64, requires_grad=True)

optimizer_q10 = torch.optim.SGD([coef_q10], lr=0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        predictions_q10 = batchX @ coef_q10
        loss_q10 = loss_fn_q10(predictions_q10, batchY.view(-1, 1))
        
        optimizer_q10.zero_grad()
        loss_q10.backward()
        optimizer_q10.step()
        
with torch.no_grad():
    predictions_q10 = testX @ coef_q10
    mse_q10 = loss_fn_q10(predictions_q10, testY.view(-1, 1)).item()

mse_q10

29.05854692548551